# Type et méthodes

JULIA possède un système de type et de méthode qui lui confère une approche objet. Le types sont organisés suivant un hiérarchie comme on peut le voir sur l'arborescence partielle ci-dessous

(arborescence générée à l'aide de https://github.com/tanmaykm/julia_types/blob/master/julia_types.jl)